# Difference-in-differences with variation in treatment timing

Prof. Daniel de Abreu Pereira Uhr

## Conteúdo

* Borusyak, Jaravel e Spiess (2023)
  * Aplicação do estimador de Borusyak, Jaravel e Spiess (2023) em Python
* Gardner (2021)
  * Aplicação do estimador de Gardner (2021) em Python
* Sun & Abraham (2021)
  * Aplicação do estimador de Sun & Abraham (2021) em Python
* Overview da literatura sobre DiD

## Referências

**Principais:**
* Goodman-Bacon, A. (2021). Difference-in-differences with variation in treatment timing. Journal of Econometrics. https://doi.org/10.1016/j.jeconom.2021.03.014
* Callaway, B. and Sant'Anna, P. H. C. (2021). Difference-in-Differences with multiple time periods. Journal of Econometrics. https://doi.org/10.1016/j.jeconom.2020.12.001
* Sun, L., & Abraham, M. (2021). Estimating Dynamic Treatment Effects in Event Studies with Heterogeneous Treatment Effects. Journal of Econometrics, 222(2), 1011-1033. https://doi.org/10.1016/j.jeconom.2020.09.006 
* Stevenson, Betsey, Wolfers, Justin, 2006. Bargaining in the shadow of the law: Divorce laws and family distress. Q. J. Econ. 121 (1), 267–288.


**Complementares:**
* https://bcallaway11.github.io/did/articles/did-basics.html 
* Borusyak, K.; Jaravel, X. and Spiess, J. (2023). Revisiting Event Study Designs: Robust and Efficient Estimation. arXiv: https://arxiv.org/pdf/2108.12419.pdf
* Clément deChaisemartin, Xavier d’Haultfoeuille. (2022) Difference-in-Differences Estimators of Intertemporal Treatment Effects. hal-03873903
* Roth et al. (2022) What’s Trending in Difference-in-Differences? A Synthesis of the Recent Econometrics Literature. https://www.jonathandroth.com/assets/files/DiD_Review_Paper.pdf
* Marcus, R., & Sant'Anna, P. H. C. (2020). The Role of Parallel Trends in Event Study Settings: An Application to Environmental Economics. Journal of the Association of Environmental and Resource Economists, 7(6), 1163-1199. https://doi.org/10.1086/710174
* Marcus and Sant’Anna (2020), “The Role of Parallel Trends in Event Study Settings: An Application to Environmental Economics,” JAERE
* Schmidheiny, K., & Siegloch, S. (2023). On event studies and distributed-lags in two-way fixed effects models: Identification, equivalence, and generalization. Journal of Applied Econometrics, 1- 19. https://doi.org/10.1002/jae.2971



**Observações:** O material apresentado aqui é uma adaptação do material de aula do Prof. Daniel de Abreu Pereira Uhr, e não deve ser utilizado para fins comerciais. O material é disponibilizado para fins educacionais e de pesquisa, e não deve ser reproduzido sem a devida autorização do autor. Este material pode conter erros e imprecisões. O autor não se responsabiliza por quaisquer danos ou prejuízos decorrentes do uso deste material. O uso deste material é de responsabilidade exclusiva do usuário. Caso você encontre erros ou imprecisões neste material, por favor, entre em contato com o autor para que possam ser corrigidos. O autor agradece qualquer *feedback* ou sugestão de melhoria.

---